## Doing

## To Do
 - [ ] add percentage change columns to normalize data, only challenge is counties with small populations could have a large percentage but not be as news worthy
 - [ ] pull data from JHU
 - [ ] pull data directly from usafacts 
 - [ ] Allow run in COLAB and save to Google Drive

## Done
 - [x] x day rolling averages
 - [x] ensure proper data types in df_combined_tidy
 - [x] make tidy data one row per observation (observations are by date)
 - [x] rename variable column 'Date' in df_combined_tidy


In [1]:
import pandas as pd
import urllib3 as urllib
import json
import math
from datetime import datetime
from IPython.core.display import HTML


import IPython.display
import altair as alt

pd.options.display.max_columns = None # Used when you have a lot of columns and want them all displayed

In [2]:
jhu_confirmed_df= pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')

# PICK ONE AND COMMENT OUT THE OTHER (local or GIT)

# READ from Local Files
# usafacts_confirmed_df= pd.read_csv('County-Data/covid_confirmed_usafacts.csv')
# usafacts_population_df= pd.read_csv('County-Data/covid_county_population_usafacts.csv')

0# READ from GIT Files - Really this should be read from source that is updated regularly
usafacts_confirmed_df= pd.read_csv('https://github.com/JaviThomas/Covid-County-LTC-Analysis/blob/master/County-Analysis/County-Data/covid_confirmed_usafacts.csv?raw=true')
usafacts_population_df= pd.read_csv('https://github.com/JaviThomas/Covid-County-LTC-Analysis/blob/master/County-Analysis/County-Data/covid_county_population_usafacts.csv?raw=true')

# 2 - Merge Confirmed Cases data with Population Data
Good Article on how to combine data sources [Combining Dataset](https://jakevdp.github.io/PythonDataScienceHandbook/03.06-concat-and-append.html)

In [3]:
df_confirmed = usafacts_confirmed_df[usafacts_confirmed_df['countyFIPS'] > 2].copy()
df_population = usafacts_population_df[usafacts_population_df['countyFIPS'] > 2].copy()

# Set Indices to facilitate pandas concat
df_population.set_index('countyFIPS', inplace=True)
df_confirmed.set_index('countyFIPS', inplace=True)

df_confirmed_combined= pd.concat ([df_confirmed, df_population['population']], axis=1).copy()
df_confirmed_combined.reset_index(inplace=True)
# REMOVE "Counties" where population is ZERO
df_cases_clean = df_confirmed_combined[df_confirmed_combined.population != 0]
df_cases_clean.head(5)

,countyFIPS,County Name,State,stateFIPS,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20,5/20/20,5/21/20,5/22/20,5/23/20,5/24/20,5/25/20,5/26/20,5/27/20,5/28/20,5/29/20,5/30/20,5/31/20,6/1/20,6/2/20,6/3/20,6/4/20,6/5/20,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20,6/12/20,6/13/20,6/14/20,6/15/20,6/16/20,6/17/20,6/18/20,6/19/20,6/20/20,6/21/20,6/22/20,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20,7/3/20,7/4/20,7/5/20,7/6/20,7/7/20,7/8/20,7/9/20,7/10/20,7/11/20,7/12/20,7/13/20,7/14/20,7/15/20,7/16/20,7/17/20,7/18/20,7/19/20,7/20/20,7/21/20,7/22/20,7/23/20,7/24/20,7/25/20,7/26/20,7/27/20,7/28/20,7/29/20,7/30/20,7/31/20,8/1/20,8/2/20,8/3/20,8/4/20,8/5/20,8/6/20,8/7/20,8/8/20,8/9/20,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20,8/19/20,8/20/20,8/21/20,8/22/20,8/23/20,8/24/20,8/25/20,8/26/20,8/27/20,8/28/20,8/29/20,8/30/20,8/31/20,9/1/20,9/2/20,9/3/20,9/4/20,9/5/20,9/6/20,9/7/20,9/8/20,9/9/20,9/10/20,9/11/20,9/12/20,9/13/20,9/14/20,9/15/20,9/16/20,9/17/20,9/18/20,9/19/20,9/20/20,9/21/20,9/22/20,9/23/20,9/24/20,9/25/20,9/26/20,9/27/20,9/28/20,9/29/20,9/30/20,10/1/20,10/2/20,10/3/20,10/4/20,10/5/20,10/6/20,10/7/20,10/8/20,10/9/20,10/10/20,10/11/20,10/12/20,10/13/20,10/14/20,10/15/20,10/16/20,10/17/20,10/18/20,10/19/20,10/20/20,10/21/20,10/22/20,10/23/20,10/24/20,10/25/20,10/26/20,10/27/20,10/28/20,10/29/20,10/30/20,10/31/20,11/1/20,11/2/20,11/3/20,11/4/20,11/5/20,11/6/20,11/7/20,11/8/20,11/9/20,11/10/20,11/11/20,11/12/20,11/13/20,11/14/20,11/15/20,11/16/20,11/17/20,11/18/20,11/19/20,11/20/20,11/21/20,11/22/20,11/23/20,11/24/20,11/25/20,11/26/20,11/27/20,11/28/20,11/29/20,11/30/20,12/1/20,12/2/20,12/3/20,12/4/20,12/5/20,12/6/20,12/7/20,12/8/20,12/9/20,12/10/20,12/11/20,12/12/20,12/13/20,12/14/20,12/15/20,12/16/20,12/17/20,12/18/20,12/19/20,12/20/20,12/21/20,12/22/20,12/23/20,12/24/20,12/25/20,12/26/20,12/27/20,12/28/20,12/29/20,12/30/20,12/31/20,1/1/21,1/2/21,1/3/21,1/4/21,1/5/21,1/6/21,1/7/21,1/8/21,1/9/21,1/10/21,1/11/21,1/12/21,1/13/21,1/14/21,1/15/21,1/16/21,1/17/21,1/18/21,1/19/21,1/20/21,1/21/21,1/22/21,1/23/21,1/24/21,population
0,1001,Autauga County,AL,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,4,6,6,6,6,7,7,10,10,12,12,12,12,12,12,17,17,19,19,19,23,25,25,25,25,27,28,30,32,33,36,37,37,39,40,42,42,42,45,48,53,53,58,61,67,68,74,84,91,93,103,103,110,110,120,127,136,147,149,155,159,173,189,192,205,212,216,220,233,238,239,241,248,259,265,272,282,295,312,323,331,357,368,373,375,400,411,431,434,442,453,469,479,488,498,503,527,537,553,561,568,591,615,618,644,651,661,670,684,706,728,746,756,780,789,827,842,857,865,886,905,921,932,942,965,974,974,1002,1015,1030,1052,1066,1073,1073,1096,1113,1134,1215,1215,1215,1241,1250,1252,1262,1273,1274,1291,1293,1293,1293,1322,1324,1351,1355,1366,1377,1389,1400,1438,1442,1452,1452,1466,1475,1492,1498,1504,1508,1522,1544,1551,1565,1576,1585,1601,1619,1624,1664,1673,1690,1691,1714,1715,1715,1757,1764,1773,1785,1787,1791,1798,1805,1818,1828,1831,1839,1852,1863,1882,1898,1905,1911,1924,1928,1949,1966,1983,1989,1999

# 2 - Make the data "tidy" to facilitate analysis and plotting
To process data it is best if it is in [tidy data](https://vita.had.co.nz/papers/tidy-data.html)  which means each row is an observation and the columns are the dimensions and metrics of that observation.

In [4]:
# date_cols = list(df_combined_clean.columns[3:-2])
id_cols = ['County Name', 'State', 'countyFIPS', 'stateFIPS', 'population'] # these are the dimensions of the data

df_cases_tidy = df_cases_clean.melt(id_vars =id_cols, value_vars = df_cases_clean.columns[4:-2], var_name='Date', value_name='Cases')

# Convert the Date column which is in a string format to a python datetime datatype.  This facilitates sorting and date differencing
df_cases_tidy['Date'] = pd.to_datetime(df_cases_tidy['Date'], format='%m/%d/%y') 

display(df_cases_tidy.dtypes) # Display data types of each column in the Dataframe
df_cases_tidy

County Name            object
State                  object
countyFIPS              int64
stateFIPS               int64
population              int64
Date           datetime64[ns]
Cases                   int64
dtype: object

,County Name,State,countyFIPS,stateFIPS,population,Date,Cases
0,Autauga County,AL,1001,1,55869,2020-01-22,0
1,Baldwin County,AL,1003,1,223234,2020-01-22,0
2,Barbour County,AL,1005,1,24686,2020-01-22,0
3,Bibb County,AL,1007,1,22394,2020-01-22,0
4,Blount County,AL,1009,1,57826,2020-01-22,0
...,...,...,...,...,...,...,...
1156251,Sweetwater County,WY,56037,56,42343,2021-01-23,3384
1156252,Teton County,WY,56039,56,23464,2021-01-23,2870
1156253,Uinta County,WY,56041,56,20226,2021-01-23,1887
1156254,Washakie County,WY,56043,56,7805,2021-01-23,853


In [5]:
%%time
# Sort data before save
df_cases_tidy.sort_values(by=['countyFIPS', 'Date'], inplace=True)
#Save to CSV file
#df_cases_tidy.to_csv('County-Data/usafacts_cases_tidy.csv') # TODO - Decide at what checkpoint to save

Wall time: 486 ms


# Step 2 - Get Rolling Averages

In [6]:
%%time
first_time = True
# set temporarily to suppress "SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame"
pd.set_option('mode.chained_assignment', None) 

# Processing the data for all states takes ~4-5 minutes. That means slow feedback when debugging and trying to add new features to the data
# So we do it for one State to get fast feedback.  Once satisfied we change to process data for all states.

# COMMENT THE Following 3 lines and UnComment out line following with for loop to get a dataframe with all states
state = 'FL'
mydf = df_cases_tidy[(df_cases_tidy['State'] == state)]
for countyFIPS in mydf['countyFIPS'].unique():

# UNCOMMENT below line (and comment above lines) to get data for all states
# for countyFIPS in df_cases_tidy['countyFIPS'].unique():  # COMMENT this line if using single state filter for faster feedback

    tmp_df = df_cases_tidy[df_cases_tidy['countyFIPS'] == countyFIPS]
#     tmp_df['7day'] = tmp_df.rolling(7).mean()['Cases']
    tmp_df['NewCases'] = tmp_df['Cases'] - tmp_df['Cases'].shift(1)
    tmp_df['7day_avg'] = tmp_df['NewCases'].rolling(7).mean().round(1)
    tmp_df['14day_avg'] = tmp_df['NewCases'].rolling(14).mean().round(1)
    tmp_df['28day_avg'] = tmp_df['NewCases'].rolling(28).mean().round(1)


    if first_time:
        df_cases_7day_tidy = tmp_df.copy()
        first_time = False
        print('First')
    else:
        df_cases_7day_tidy = df_cases_7day_tidy.append(tmp_df)

pd.set_option('mode.chained_assignment', 'warn') # Turn off warning suppresion

First
Wall time: 619 ms


In [7]:
df_cases_7day_tidy.tail(10)  # Did this to ensure my calculations worked. Change to head and see what happens

,County Name,State,countyFIPS,stateFIPS,population,Date,Cases,NewCases,7day_avg,14day_avg,28day_avg
1125222,Washington County,FL,12133,12,25473,2021-01-14,2226,29.0,15.4,18.2,17.2
1128364,Washington County,FL,12133,12,25473,2021-01-15,2244,18.0,15.1,19.5,17.4
1131506,Washington County,FL,12133,12,25473,2021-01-16,2261,17.0,14.7,17.4,17.4
1134648,Washington County,FL,12133,12,25473,2021-01-17,2263,2.0,14.6,16.6,17.3
1137790,Washington County,FL,12133,12,25473,2021-01-18,2273,10.0,15.3,16.1,17.0
1140932,Washington County,FL,12133,12,25473,2021-01-19,2282,9.0,14.7,16.1,17.0
1144074,Washington County,FL,12133,12,25473,2021-01-20,2295,13.0,14.0,14.9,16.6
1147216,Washington County,FL,12133,12,25473,2021-01-21,2315,20.0,12.7,14.1,16.0
1150358,Washington County,FL,12133,12,25473,2021-01-22,2326,11.0,11.7,13.4,16.4
1153500,Washington County,FL,12133,12,25473,2021-01-23,2345,19.0,12.0,13.4,16.0


In [8]:
%%time
# Save data - optional decide at what point you want to save a copy for sharing with others
df_cases_7day_tidy.to_csv('df_cases_7day_tidy.tmp.csv')

Wall time: 335 ms


# 3 - Add %Diff vs yesterday, 7 days ago, 14 days ago, 28 days ago

In [9]:
# Parameterize differecing columns by date

# Dictionary of columns 
diffCols = [{'col2Diff': 'NewCases', 'daysAgo2Diff': 7},
            {'col2Diff': '7day_avg', 'daysAgo2Diff': 7},
            {'col2Diff': '14day_avg', 'daysAgo2Diff': 7},
            {'col2Diff': '28day_avg', 'daysAgo2Diff': 7}]

for item in diffCols:
    newColName = item['col2Diff'] + '_' + str(item['daysAgo2Diff']) + 'DayDiff'
    df_cases_7day_tidy[newColName] = (df_cases_7day_tidy[item['col2Diff']] - df_cases_7day_tidy[item['col2Diff']].shift(item['daysAgo2Diff'])) 

df_cases_7day_tidy.sort_values(by=['countyFIPS', 'Date']).tail(10)

,County Name,State,countyFIPS,stateFIPS,population,Date,Cases,NewCases,7day_avg,14day_avg,28day_avg,NewCases_7DayDiff,7day_avg_7DayDiff,14day_avg_7DayDiff,28day_avg_7DayDiff
1125222,Washington County,FL,12133,12,25473,2021-01-14,2226,29.0,15.4,18.2,17.2,-3.0,-5.6,0.3,-0.6
1128364,Washington County,FL,12133,12,25473,2021-01-15,2244,18.0,15.1,19.5,17.4,-2.0,-8.8,0.2,-0.6
1131506,Washington County,FL,12133,12,25473,2021-01-16,2261,17.0,14.7,17.4,17.4,-3.0,-5.4,-1.2,-1.0
1134648,Washington County,FL,12133,12,25473,2021-01-17,2263,2.0,14.6,16.6,17.3,-1.0,-4.1,-1.5,-0.6
1137790,Washington County,FL,12133,12,25473,2021-01-18,2273,10.0,15.3,16.1,17.0,5.0,-1.6,-2.4,-0.7
1140932,Washington County,FL,12133,12,25473,2021-01-19,2282,9.0,14.7,16.1,17.0,-4.0,-2.7,-2.5,-0.9
1144074,Washington County,FL,12133,12,25473,2021-01-20,2295,13.0,14.0,14.9,16.6,-5.0,-1.9,-3.7,-0.7
1147216,Washington County,FL,12133,12,25473,2021-01-21,2315,20.0,12.7,14.1,16.0,-9.0,-2.7,-4.1,-1.2
1150358,Washington County,FL,12133,12,25473,2021-01-22,2326,11.0,11.7,13.4,16.4,-7.0,-3.4,-6.1,-1.0
1153500,Washington County,FL,12133,12,25473,2021-01-23,2345,19.0,12.0,13.4,16.0,2.0,-2.7,-4.0,-1.4


# 4 - Get data from latest date to get top 10's

In [11]:
# Get DataFrame with only latest date
df_cases_7day_tidy_latest = df_cases_7day_tidy[df_cases_7day_tidy['Date'] == df_cases_7day_tidy['Date'].max()].copy()

# top10Fields = ['NewCases_7DayDiff', 'NewCases_14DayDiff', 'NewCases_28DayDiff']
top10Fields = [{'colName': '7day_avg_7DayDiff', 'colTitle': '<7 Day Avg.> - <7 Day Avg. from 7 days ago>'},
               {'colName': '14day_avg_7DayDiff', 'colTitle': '<14 Day Avg.>. - <14 Day Avg. from 7 days ago>'},
               {'colName': '28day_avg_7DayDiff', 'colTitle': '<28 Day Avg.> - <28 Day Avg. from 7 days ago>'}]

cols2show = ['County Name', 'State', 'countyFIPS', 'stateFIPS', 'population', 'Date', 'Cases', 'NewCases', '7day_avg', '14day_avg', '28day_avg']
for field in top10Fields:
    cols2show.append(field['colName'])
    display(HTML('<h2>Top 10 Counties with <b>&#8607;</b> Rising Rates  <b>&#8607;</b> by: ' + field['colTitle'] + '</h2>'))
    display(df_cases_7day_tidy_latest.sort_values(by=field['colName'], ascending=False)[cols2show].head(10)) # Top 10 Rising
    
    display(HTML('<h2>Top 10 Counties with <b>&#8609;</b> Falling Rates <b>&#8609;</b> by: ' + field['colTitle'] + '</h2>'))
    display(df_cases_7day_tidy_latest.sort_values(by=field['colName'])[cols2show].head(10)) # Top 10 Falling
    
    cols2show = ['County Name', 'State', 'countyFIPS', 'stateFIPS', 'population', 'Date', 'Cases', 'NewCases', '7day_avg', '14day_avg', '28day_avg']


,County Name,State,countyFIPS,stateFIPS,population,Date,Cases,NewCases,7day_avg,14day_avg,28day_avg,7day_avg_7DayDiff
1153436,Bay County,FL,12005,12,174705,2021-01-23,15159,115.0,172.9,145.4,137.8,54.9
1153469,Leon County,FL,12073,12,293582,2021-01-23,24991,253.0,228.1,215.8,232.2,24.7
1153472,Madison County,FL,12079,12,18493,2021-01-23,1770,14.0,10.4,9.4,9.3,2.1
1153465,Jefferson County,FL,12065,12,14246,2021-01-23,1241,8.0,9.3,9.3,10.6,0.0
1153453,Gilchrist County,FL,12041,12,18582,2021-01-23,1160,9.0,5.1,5.4,6.6,-0.5
1153473,Manatee County,FL,12081,12,403253,2021-01-23,27224,234.0,191.9,192.4,212.4,-1.1
1153440,Calhoun County,FL,12013,12,14105,2021-01-23,1485,3.0,6.7,7.4,11.8,-1.3
1153466,Lafayette County,FL,12067,12,8422,2021-01-23,1511,2.0,3.0,3.6,3.5,-1.3
1153454,Glades County,FL,12043,12,13811,2021-01-23,854,4.0,3.0,3.6,3.4,-1.3
1153456,Hamilton County,FL,12047,12,14428,2021-01-23,1465,5.0,7.6,8.4,9.0,-1.5


,County Name,State,countyFIPS,stateFIPS,population,Date,Cases,NewCases,7day_avg,14day_avg,28day_avg,7day_avg_7DayDiff
1153476,Miami-Dade County,FL,12086,12,2716940,2021-01-23,357038,2334.0,2170.9,2341.3,2513.4,-340.8
1153448,Duval County,FL,12031,12,957755,2021-01-23,78161,420.0,515.4,599.1,720.1,-167.5
1153481,Orange County,FL,12095,12,1393452,2021-01-23,97842,994.0,771.9,854.8,899.2,-165.8
1153461,Hillsborough County,FL,12057,12,1471968,2021-01-23,96338,861.0,672.3,748.8,798.5,-153.0
1153439,Broward County,FL,12011,12,1952778,2021-01-23,165045,1149.0,1065.0,1134.1,1173.3,-138.1
1153485,Pinellas County,FL,12103,12,974996,2021-01-23,56564,317.0,383.1,433.8,485.4,-101.3
1153483,Palm Beach County,FL,12099,12,1496770,2021-01-23,101831,812.0,726.9,770.1,785.8,-86.5
1153486,Polk County,FL,12105,12,724777,2021-01-23,47795,541.0,367.3,409.9,442.9,-85.3
1153474,Marion County,FL,12083,12,365579,2021-01-23,23209,159.0,168.4,206.6,224.6,-76.3
1153468,Lee County,FL,12071,12,770577,2021-01-23,50278,388.0,353.6,389.9,418.0,-72.7


,County Name,State,countyFIPS,stateFIPS,population,Date,Cases,NewCases,7day_avg,14day_avg,28day_avg,14day_avg_7DayDiff
1153436,Bay County,FL,12005,12,174705,2021-01-23,15159,115.0,172.9,145.4,137.8,15.6
1153472,Madison County,FL,12079,12,18493,2021-01-23,1770,14.0,10.4,9.4,9.3,0.5
1153451,Franklin County,FL,12037,12,12125,2021-01-23,1172,8.0,9.6,11.7,10.6,0.1
1153456,Hamilton County,FL,12047,12,14428,2021-01-23,1465,5.0,7.6,8.4,9.0,-0.1
1153457,Hardee County,FL,12049,12,26937,2021-01-23,2487,15.0,14.3,16.8,15.8,-0.4
1153466,Lafayette County,FL,12067,12,8422,2021-01-23,1511,2.0,3.0,3.6,3.5,-0.4
1153454,Glades County,FL,12043,12,13811,2021-01-23,854,4.0,3.0,3.6,3.4,-0.5
1153453,Gilchrist County,FL,12041,12,18582,2021-01-23,1160,9.0,5.1,5.4,6.6,-0.7
1153447,Dixie County,FL,12029,12,16826,2021-01-23,1357,17.0,6.7,10.4,9.8,-0.9
1153465,Jefferson County,FL,12065,12,14246,2021-01-23,1241,8.0,9.3,9.3,10.6,-1.0


,County Name,State,countyFIPS,stateFIPS,population,Date,Cases,NewCases,7day_avg,14day_avg,28day_avg,14day_avg_7DayDiff
1153476,Miami-Dade County,FL,12086,12,2716940,2021-01-23,357038,2334.0,2170.9,2341.3,2513.4,-348.3
1153448,Duval County,FL,12031,12,957755,2021-01-23,78161,420.0,515.4,599.1,720.1,-234.1
1153481,Orange County,FL,12095,12,1393452,2021-01-23,97842,994.0,771.9,854.8,899.2,-140.0
1153461,Hillsborough County,FL,12057,12,1471968,2021-01-23,96338,861.0,672.3,748.8,798.5,-119.8
1153439,Broward County,FL,12011,12,1952778,2021-01-23,165045,1149.0,1065.0,1134.1,1173.3,-114.5
1153485,Pinellas County,FL,12103,12,974996,2021-01-23,56564,317.0,383.1,433.8,485.4,-107.1
1153486,Polk County,FL,12105,12,724777,2021-01-23,47795,541.0,367.3,409.9,442.9,-76.5
1153468,Lee County,FL,12071,12,770577,2021-01-23,50278,388.0,353.6,389.9,418.0,-73.7
1153483,Palm Beach County,FL,12099,12,1496770,2021-01-23,101831,812.0,726.9,770.1,785.8,-70.7
1153491,Sarasota County,FL,12115,12,433742,2021-01-23,23718,115.0,171.4,192.9,213.2,-55.9


,County Name,State,countyFIPS,stateFIPS,population,Date,Cases,NewCases,7day_avg,14day_avg,28day_avg,28day_avg_7DayDiff
1153439,Broward County,FL,12011,12,1952778,2021-01-23,165045,1149.0,1065.0,1134.1,1173.3,59.4
1153483,Palm Beach County,FL,12099,12,1496770,2021-01-23,101831,812.0,726.9,770.1,785.8,50.1
1153481,Orange County,FL,12095,12,1393452,2021-01-23,97842,994.0,771.9,854.8,899.2,39.5
1153469,Leon County,FL,12073,12,293582,2021-01-23,24991,253.0,228.1,215.8,232.2,28.3
1153476,Miami-Dade County,FL,12086,12,2716940,2021-01-23,357038,2334.0,2170.9,2341.3,2513.4,22.5
1153438,Brevard County,FL,12009,12,601942,2021-01-23,27542,297.0,244.4,265.4,267.6,20.2
1153449,Escambia County,FL,12033,12,318316,2021-01-23,29045,288.0,257.6,278.0,277.8,20.1
1153468,Lee County,FL,12071,12,770577,2021-01-23,50278,388.0,353.6,389.9,418.0,15.0
1153436,Bay County,FL,12005,12,174705,2021-01-23,15159,115.0,172.9,145.4,137.8,14.2
1153497,Volusia County,FL,12127,12,553284,2021-01-23,29051,201.0,257.3,275.1,280.1,13.8


,County Name,State,countyFIPS,stateFIPS,population,Date,Cases,NewCases,7day_avg,14day_avg,28day_avg,28day_avg_7DayDiff
1153448,Duval County,FL,12031,12,957755,2021-01-23,78161,420.0,515.4,599.1,720.1,-18.0
1153437,Bradford County,FL,12007,12,28201,2021-01-23,2580,15.0,13.4,15.2,17.5,-9.7
1153488,St. Johns County,FL,12109,12,264672,2021-01-23,17379,137.0,131.3,158.1,181.4,-7.5
1153479,Okaloosa County,FL,12091,12,210738,2021-01-23,15827,98.0,102.9,121.1,126.4,-6.7
1153443,Clay County,FL,12019,12,219252,2021-01-23,14658,84.0,107.7,122.9,141.1,-5.1
1153484,Pasco County,FL,12101,12,553947,2021-01-23,28076,211.0,213.1,243.8,255.6,-4.9
1153441,Charlotte County,FL,12015,12,188910,2021-01-23,9316,59.0,56.4,66.7,82.7,-4.7
1153445,Columbia County,FL,12023,12,71686,2021-01-23,7023,35.0,31.9,37.4,44.7,-4.1
1153462,Holmes County,FL,12059,12,19617,2021-01-23,2014,10.0,7.1,9.8,15.4,-3.4
1153494,Suwannee County,FL,12121,12,44417,2021-01-23,4716,18.0,20.1,22.2,30.0,-2.4


# Display Charts for Counties to compare different measures to detect trends

In [19]:
date_time_str = '2020-10-01'
date_time_obj = datetime.strptime(date_time_str, '%Y-%m-%d')



top10Fields = ['7day_avg_7DayDiff', '14day_avg_7DayDiff', '28day_avg_7DayDiff']
top10Fields = [{'colName': '7day_avg_7DayDiff', 'colTitle': '<7 Day Avg.> - <7 Day Avg. from 7 days ago>'},
               {'colName': '14day_avg_7DayDiff', 'colTitle': '<14 Day Avg.>. - <14 Day Avg. from 7 days ago>'},
               {'colName': '28day_avg_7DayDiff', 'colTitle': '<28 Day Avg.> - <28 Day Avg. from 7 days ago>'}]

# Goal is to print the 3 charts (one per field) in a row per FIPS
topN = 3
for field in top10Fields:
    display(HTML('<H1>Top ' + str(topN) + ' Counties by: ' + field['colTitle'] + '</H1><BR>'))

    for countyFIPS in df_cases_7day_tidy_latest.sort_values(by=field['colName'], ascending=False)['countyFIPS'].head(topN):
        charts = []
        for field in top10Fields:
            state_df = df_cases_7day_tidy[(df_cases_7day_tidy['countyFIPS'] == countyFIPS) & (df_cases_7day_tidy['Date'] >= date_time_obj )]
            row = state_df.iloc[0]
            #display(row)
            chart_title = row['State'] + ': ' + row['County Name'] + ' - Population: ' + f"{row['population']:,}"
            
            chart1 = alt.Chart(state_df).mark_bar().encode(
                                 x=alt.X('yearmonthdate(Date):T', title='date', axis=alt.Axis(labelSeparation=2)),
                                 y=alt.Y(field['colName']+':Q', title=field['colTitle']),
                                 ).properties( width=400, height=400, title=chart_title)
            charts.append(chart1)
        display(charts[0] | charts[1] | charts[2])
    display(HTML('<HR>'))


alt.HConcatChart(...)

alt.HConcatChart(...)

alt.HConcatChart(...)

alt.HConcatChart(...)

alt.HConcatChart(...)

alt.HConcatChart(...)

alt.HConcatChart(...)

alt.HConcatChart(...)

alt.HConcatChart(...)

# Play Area - WARNING this code MAY not work as I would expect

In [13]:
row = state_df.iloc[0]
#display(row)
row['State'] + ': ' + row['County Name'] + ', Population: ' + f"{row['population']:,}"
#f"{row['population']:,}"

'FL: Orange County, Population: 1,393,452'

In [14]:
# OKAY I didn't make the data "Fully tidy"  if I did I could use the facet option instead of for loops
# ====================================================================================================

from vega_datasets import data
iris = data.iris()

alt.Chart(iris).mark_point().encode(
    x='petalLength:Q',
    y='petalWidth:Q',
    color='species:N'
).properties(
    width=180,
    height=180
).facet(
    facet='species:N',
    columns=2
)

alt.FacetChart(...)

In [15]:
iris

,sepalLength,sepalWidth,petalLength,petalWidth,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


# PLAY AREA